## Convert f2x_swapi.h (f2x_swapi_flow.h) to xxxx_c.h
- remove all /\* ... \*/ 
### To do manually in xxxx_c.h
- remove some defines
- remove #if 0
- Replace macros with real numbers
### Save new file as xxxx_d.h


In [2]:
def process_org_file(fn):
    f = open(fn, "r")
    lines = f.readlines()

    clines = list()
    rmask = 0
    for line in lines:
        a = line.split()
        if (len(a)>0) and (len(a[0])>1) and (a[0][0:2] == "/*"):
            rmask = 1
        if rmask == 1:
            if "*/" in line:
                rmask = 0
            continue
        clines.append(line)

    fw = open(fn[:-2]+"_c"+fn[-2:],"w")
    for line in clines:
        fw.write(line)

process_org_file("f2x_swapi.h")
process_org_file("f2x_swapi_flow.h")

## Process f2x_swapi_d.h and f2x_swapi_flow_d.h
### From typedef to generate dictionary of types / variables / structures / enums
- the "{" of typedef must be a line wihout anything else. 
- the "}" of typedef must be seperated from the type name.






In [2]:
def process_d_file(fn, struct, enum, enum_var_ref):
    f = open(fn,"r")
    lines = f.readlines()

    ty = ["u64", "u32", "u16", "u8", "i32", "i16", "i8", "ur", "ltq_bool_t", "int", "char"]
    s = 0
    e = 0
    i = 0
    for line in lines:
        i = i + 1
        a = line.split()
        if ((s == 0) and (e == 0)) and len(a) == 1:
            print ("Something Wrong 1 at line: ", line, i)
        if (len(a)>1) and (a[0] == "typedef"):
            if (a[1] == "struct"):
                s = 1
                l = list()
                ref = list()
            if (a[1] == "enum"):
                e = 1
                l = dict()
            continue
        if (s == 1) or (e == 1):
            if (len(a) == 0):
                continue
            if (a[0] == "{"):
                continue
            if (a[0] in ty) or (a[0][-2:] == "_t"): 
                l.append([a[0], a[1].replace(";", "")]) 
                if a[0][-2:] == "_t" and (a[0] not in ty):
                    ref.append([a[0], a[1].replace(";", "")]) 
            elif (a[0] != "}" and e == 1 and a[1] == "="):
                l[a[2].replace(",", "")] = a[0]
            elif (a[0] == "}"):
                if s == 1:
                    struct[a[1][:-1]] = l
                    for r in ref:
                        enum_var_ref[a[1].replace(";", "") + r[1]] = r[0]
                    s = 0
                if e == 1:
                    enum[a[1][:-1]] = l
                    e = 0
            else:
                print ("Something Wrong 2 at line: ", line, i)


In [3]:
struct = dict()
enum = dict()
enum_var_ref = dict()

process_d_file("f2x_swapi_flow_d.h", struct, enum, enum_var_ref)
process_d_file("f2x_swapi_d.h", struct, enum, enum_var_ref)



In [4]:
print (len(enum_var_ref))
enum_var_ref

66


{'GSW_irq_teIrqSrc': 'GSW_irqSrc_t',
 'GSW_PCE_pattern_teDstIP_Select': 'GSW_PCE_IP_t',
 'GSW_PCE_pattern_tnDstIP': 'GSW_IP_t',
 'GSW_PCE_pattern_teSrcIP_Select': 'GSW_PCE_IP_t',
 'GSW_PCE_pattern_tnSrcIP': 'GSW_IP_t',
 'GSW_PCE_action_teTrafficClassAction': 'GSW_PCE_ActionTrafficClass_t',
 'GSW_PCE_action_teSnoopingTypeAction': 'GSW_PCE_ActionIGMP_Snoop_t',
 'GSW_PCE_action_teLearningAction': 'GSW_PCE_ActionLearning_t',
 'GSW_PCE_action_teIrqAction': 'GSW_PCE_ActionIrq_t',
 'GSW_PCE_action_teCrossStateAction': 'GSW_PCE_ActionCrossState_t',
 'GSW_PCE_action_teCritFrameAction': 'GSW_PCE_ActionCriticalFrame_t',
 'GSW_PCE_action_teTimestampAction': 'GSW_PCE_ActionTimestamp_t',
 'GSW_PCE_action_tePortMapAction': 'GSW_PCE_ActionPortmap_t',
 'GSW_PCE_action_teMeterAction': 'GSW_PCE_ActionMeter_t',
 'GSW_PCE_action_teVLAN_Action': 'GSW_PCE_ActionVLAN_t',
 'GSW_PCE_action_teSVLAN_Action': 'GSW_PCE_ActionVLAN_t',
 'GSW_PCE_action_teVLAN_CrossAction': 'GSW_PCE_ActionCrossVLAN_t',
 'GSW_PCE_rule_

In [5]:
print (len(enum))
enum

47


{'GSW_irqSrc_t': {'0': 'GSW_IRQ_WOL',
  '1': 'GSW_IRQ_LIMIT_ALERT',
  '2': 'GSW_IRQ_LOCK_ALERT'},
 'GSW_PCE_IP_t': {'0': 'GSW_PCE_IP_DISABLED',
  '1': 'GSW_PCE_IP_V4',
  '2': 'GSW_PCE_IP_V6'},
 'GSW_PCE_ActionIGMP_Snoop_t': {'0': 'GSW_PCE_ACTION_IGMP_SNOOP_DISABLE',
  '1': 'GSW_PCE_ACTION_IGMP_SNOOP_REGULAR',
  '2': 'GSW_PCE_ACTION_IGMP_SNOOP_REPORT',
  '3': 'GSW_PCE_ACTION_IGMP_SNOOP_LEAVE',
  '4': 'GSW_PCE_ACTION_IGMP_SNOOP_AD',
  '5': 'GSW_PCE_ACTION_IGMP_SNOOP_QUERY',
  '6': 'GSW_PCE_ACTION_IGMP_SNOOP_QUERY_GROUP',
  '7': 'GSW_PCE_ACTION_IGMP_SNOOP_QUERY_NO_ROUTER'},
 'GSW_PCE_ActionLearning_t': {'0': 'GSW_PCE_ACTION_LEARNING_DISABLE',
  '1': 'GSW_PCE_ACTION_LEARNING_REGULAR',
  '2': 'GSW_PCE_ACTION_LEARNING_FORCE_NOT',
  '3': 'GSW_PCE_ACTION_LEARNING_FORCE'},
 'GSW_PCE_ActionMeter_t': {'0': 'GSW_PCE_ACTION_METER_DISABLE',
  '1': 'GSW_PCE_ACTION_METER_REGULAR',
  '2': 'GSW_PCE_ACTION_METER_1',
  '3': 'GSW_PCE_ACTION_METER_1_2'},
 'GSW_PCE_ActionTrafficClass_t': {'0': 'GSW_PCE_ACTIO

In [6]:
print (len(struct))
struct

82


{'GSW_register_t': [['u16', 'nRegAddr'], ['u16', 'nData']],
 'GSW_irq_t': [['u32', 'nPortId'], ['GSW_irqSrc_t', 'eIrqSrc']],
 'GSW_PCE_pattern_t': [['int', 'nIndex'],
  ['ltq_bool_t', 'bEnable'],
  ['ltq_bool_t', 'bPortIdEnable'],
  ['u8', 'nPortId'],
  ['ltq_bool_t', 'bDSCP_Enable'],
  ['u8', 'nDSCP'],
  ['ltq_bool_t', 'bPCP_Enable'],
  ['u8', 'nPCP'],
  ['ltq_bool_t', 'bSTAG_PCP_DEI_Enable'],
  ['u8', 'nSTAG_PCP_DEI'],
  ['ltq_bool_t', 'bPktLngEnable'],
  ['u16', 'nPktLng'],
  ['u16', 'nPktLngRange'],
  ['ltq_bool_t', 'bMAC_DstEnable'],
  ['u8', 'nMAC_Dst[6]'],
  ['u16', 'nMAC_DstMask'],
  ['ltq_bool_t', 'bMAC_SrcEnable'],
  ['u8', 'nMAC_Src[6]'],
  ['u16', 'nMAC_SrcMask'],
  ['ltq_bool_t', 'bAppDataMSB_Enable'],
  ['u16', 'nAppDataMSB'],
  ['ltq_bool_t', 'bAppMaskRangeMSB_Select'],
  ['u16', 'nAppMaskRangeMSB'],
  ['ltq_bool_t', 'bAppDataLSB_Enable'],
  ['u16', 'nAppDataLSB'],
  ['ltq_bool_t', 'bAppMaskRangeLSB_Select'],
  ['u16', 'nAppMaskRangeLSB'],
  ['GSW_PCE_IP_t', 'eDstIP_Sele

In [7]:
s = ""
for key in enum:
    s = s + key + ", "
s

'GSW_irqSrc_t, GSW_PCE_IP_t, GSW_PCE_ActionIGMP_Snoop_t, GSW_PCE_ActionLearning_t, GSW_PCE_ActionMeter_t, GSW_PCE_ActionTrafficClass_t, GSW_PCE_ActionIrq_t, GSW_PCE_ActionCrossState_t, GSW_PCE_ActionCriticalFrame_t, GSW_PCE_ActionTimestamp_t, GSW_PCE_ActionPortmap_t, GSW_PCE_ActionVLAN_t, GSW_PCE_ActionCrossVLAN_t, GSW_resetMode_t, GSW_portForward_t, GSW_STP_PortState_t, GSW_8021X_portState_t, GSW_VLAN_Admit_t, GSW_VLAN_MemberViolation_t, GSW_Qos_ingressRemarking_t, GSW_QoS_DropPrecedence_t, GSW_QoS_ClassSelect_t, GSW_QoS_WRED_ThreshMode_t, GSW_QoS_WRED_Profile_t, GSW_direction_t, GSW_QoS_Scheduler_t, GSW_multicastSnoopMode_t, GSW_multicastReportSuppression_t, GSW_IGMP_MemberMode_t, GSW_HW_InitMode_t, GSW_ageTimer_t, GSW_portSpeed_t, GSW_portDuplex_t, GSW_portLink_t, GSW_capType_t, GSW_portEnable_t, GSW_portMonitor_t, GSW_portFlow_t, GSW_CPU_SpecialTagEthType_t, GSW_CPU_HeaderMode_t, GSW_CPU_Pause_t, GSW_MII_Mode_t, GSW_MII_Type_t, GSW_clkMode_t, GSW_return_t, GSW_RMON_type_t, GSW_RMON

## Process gsw_api_func.h to gsw_api_func_c.h
- remove //* ... *//
- every line for one function
- remove "(", ")", ",", ";"
## Manually convert _c.h file to _d.h file
#### Remove the return character in the end of last line
#### Remove "GSW_return_t ", "void *cdev ", "*parm "
#### Remove GSW_CPU_PortExtendCfgGet(), GSW_CPU_PortExtendCfgSet() GSW_IrqGet(), GSW_IrqStatusClear() because swapi v1.0.5 doesn't support
#### Remove GSW_MmdDataRead(), GSW_MmdDataWrite() because the implementation in swapi v1.0.5 is wrong.
#### Add instruction in the end of every function
- 1st value, 
    - = 0, set funcion with param, 
    - = 1, set function without param, 
    - = 2, get function with param,
    - = 3, get function without param, but there is param structure in C code when calling the API.
    - = 4, GSW_PceRuleWrite()
    - = 5, GSW_PceRuleRead()
- for get funtion, list all the param serial numbers (in the struct, start from 0) needed when calling this function

In [79]:
process_org_file("gsw_api_func.h")

f = open("gsw_api_func_c.h", "r")
lines = f.readlines()
ls = list()
import re
for line in lines:
    if line == "\n":
        continue
    end = 0
    try:
        if (line.split())[-1][-1] == ";":
            end = 1
    except:
        print (line, line.split(), len(line))
    line = re.sub("[;,()]", " ", line)
    ls.append(line.split())
    if end == 1:
        ls[-1].append("\n")

f = open("gsw_api_func_c.h", "w")
for l in ls:
    for i in l:
        f.write(i)
        if i != "\n":
            f.write(" ")
f.flush()

NameError: name 'process_org_file' is not defined

## Generate C / Python code

In [24]:
f = open("gsw_api_func_d.h", "r")
lines = f.readlines()

fc = open("gsw_py_interface_c.c", "w")
fh = open("gsw_py_interface_c.h", "w")

print (len(lines))
l = lines[0].split()
l



129


['GSW_8021X_EAPOL_RuleGet', 'GSW_8021X_EAPOL_Rule_t', '2', '1']

In [25]:
def cg013(line, i):
    l = line.split()
    paralist = list()
    indentation = "          "
    s = "int py" + l[0] + "("
    #print (i, l[i])
    if l[i] == "0":
        #print (struct[l[1]])
        for para in struct[l[1]]:
            #print (l[1], para)
            if (para[1][-1] == "]"):    # process array
                #print (para)
                #print (l[1], struct[l[1]])
                n = para[1][para[1].find("[")+1:-1]
                name = para[1][:para[1].find("[")]
                #print ("n  = ", n, "name = ", name)                
                for nn in range(int(n)):                 
                    s = s + para[0] + " " + name + str(nn)
                    paralist.append([name + "[" + str(nn) + "]", name + str(nn)])
                    if (len(s) > 100):
                        fc.write(s)
                        fh.write(s)
                        s = ", \\\n" + indentation
                    else:
                        s = s + ", "
            else:
                s = s + para[0] + " " + para[1]
                paralist.append([para[1], para[1]])
                if (len(s) > 100):
                    fc.write(s)
                    fh.write(s)
                    s = ", \\\n" + indentation
                else:
                    s = s + ", "
        if s == (", \\\n" + indentation):
            s = ", "
        s = s[:-2]
    s = s + ")\n"
    #print (s)
    fc.write(s)
    s = s[:-1] + ";\n"
    fh.write(s)    # Function line
    s = "{\n"
    fc.write(s)    # 2nd line: {
    if l[i] == "3":
        s = "    FILE *fp;\n"    #declare file handler
        fc.write(s)
    s = "    int ret;\n"    #declare return value    
    fc.write(s)    
    if l[i] == "0" or l[i] == "3":     #declaire the param
        s = "    " + l[1] + "  param;\n\n"    
        fc.write(s)
        s = "    memset( &param, 0, sizeof("+l[1]+"));\n"
        fc.write(s)
        if l[i] == "0":    #for set API, give value of every parameter
            for p in paralist:
                s = "    param." + p[0] + " = " + p[1] + ";\n"
                fc.write(s)
    if l[i] == "0" or l[i] == "3":    # call the API
        s = "\n    ret = " + l[0] + ("(pedev0, &param);\n")    
    else:
        s = "\n    ret = " + l[0] + ("(pedev0);\n")
    fc.write(s)
    if l[i] == "3":    #save get values to file
        s = "    fp = fopen (fn, \"w+\");\n"    
        fc.write(s)
        s = "    fprintf(fp, \"%s\\r\\n\", \"" + l[0] + "\");\n"    #print the fuction name
        fc.write(s)
        for para in struct[l[1]]:
            if para[1][-1] != "]":    #none array value
                s = "    fprintf(fp, \"%s = 0x%x, %d\\r\\n\", \"" + para[1] + "\", param." + para[1] + ", param." + para[1] + ");\n"
                fc.write(s)
            else:               #print arry value to file as: name[n] = [xx, xx, xx, xx] 
                n = para[1][para[1].find("[")+1:-1]
                name = para[1][:para[1].find("[")]      
                s = "    fprintf(fp, \"%s = [\", \""  + para[1] + "\");\n"
                fc.write(s)    
                for nn in range(int(n)):
                    s = "    fprintf(fp, \" 0x%02x\", " + "param." + name + "[" + str(nn) + "]);\n"
                    fc.write(s)
                s = "    fprintf(fp, \"]\\r\\n\");\n"
                fc.write(s)
        s = "    fclose(fp);\n"
        fc.write(s)
    s = "\n    return ret;\n}\n\n"
    fc.write(s)
    return







def cg2(line):
    l = line.split()
    indentation = "          "    
    s = "int py" + l[0] + "("    #function name
    paralist = list()
    for i in l[3:]:
        #print (l[1], para)
        i = int(i)
        if (struct[l[1]][i][1][-1] == "]"):    # process array
            #print (para)
            #print (l[1], struct[l[1]])
            n = struct[l[1]][i][1][struct[l[1]][i][1].find("[")+1:-1]
            name = struct[l[1]][i][1][:struct[l[1]][i][1].find("[")]
            #print ("n  = ", n, "name = ", name)                
            for nn in range(int(n)):
                s = s + struct[l[1]][i][0] + " " + name + str(nn)
                paralist.append([name + "[" + str(nn) + "]", name + str(nn)])
                if (len(s) > 100):
                    fc.write(s)
                    fh.write(s)
                    s = ", \\\n" + indentation
                else:
                    s = s + ", "
        else:
            s = s + struct[l[1]][i][0] + " " + struct[l[1]][i][1]
            paralist.append([struct[l[1]][i][1], struct[l[1]][i][1]])
            if (len(s) > 100):
                fc.write(s)
                fh.write(s)
                s = ", \\\n" + indentation
            else:
                s = s + ", "
    if s == (", \\\n" + indentation):
        s = ", "                
    s = s[:-2]
    s = s + ")\n"
    #print (s)
    fc.write(s)
    s = s[:-1] + ";\n"
    fh.write(s)    # Function line
    s = "{\n"
    fc.write(s)    # 2nd line: {
    s = "    FILE *fp;\n"    #declare file handler
    fc.write(s)
    s = "    int ret;\n"    #declare return value    
    fc.write(s)    
    s = "    " + l[1] + "  param;\n\n"    #declaire the param
    fc.write(s)
    s = "    memset( &param, 0, sizeof("+l[1]+"));\n"
    fc.write(s)
    for p in paralist:    #give value of every input parameter
        s = "    param." + p[0] + " = " + p[1] + ";\n"
        fc.write(s)
    s = "\n    ret = " + l[0] + ("(pedev0, &param);\n\n")    #call the function
    fc.write(s)

    s = "    fp = fopen (fn, \"w+\");\n"    #save get values to file
    fc.write(s)
    s = "    fprintf(fp, \"%s\\r\\n\", \"" + l[0] + "\");\n"    #print the fuction name
    fc.write(s)
    for para in struct[l[1]]:
        if para[1][-1] != "]":    #none array value
            s = "    fprintf(fp, \"%s = 0x%x, %d\\r\\n\", \"" + para[1] + "\", param." + para[1] + ", param." + para[1] + ");\n"
            fc.write(s)
        else:               #print arry value to file as: name[n] = [xx, xx, xx, xx] 
            n = para[1][para[1].find("[")+1:-1]
            name = para[1][:para[1].find("[")]      
            s = "    fprintf(fp, \"%s = [\", \""  + para[1] + "\");\n"
            fc.write(s)    
            for nn in range(int(n)):
                s = "    fprintf(fp, \" 0x%02x\", " + "param." + name + "[" + str(nn) + "]);\n"
                fc.write(s)
            s = "    fprintf(fp, \"]\\r\\n\");\n"
            fc.write(s)
    s = "    fclose(fp);\n"
    fc.write(s)

    s = "\n    return ret;\n}\n\n"
    fc.write(s)
    return


'''
'GSW_PCE_rule_t': [['GSW_PCE_pattern_t', 'pattern'],
  ['GSW_PCE_action_t', 'action']],
'''

def cg45(line):
    return

for line in lines:
    l = line.split()
    if len(l) == 2:
        i = 1
    else:
        i = 2
    #print (l, i, len(l))
    #print (l[i])
    if l[i] == "0" or l[i] == "1" or l[i] == "3":
        #print ("calling cg01")
        cg013(line, i)
    elif l[i] == "2":
        cg2(line)
    elif l[i] == "4" or l[i] == "5":
        cg45(line)
    else:
        print ("Something WRONG at line:", line)        

fc.close()
fh.close()


## Generate Switch.py

def pyGSW_MAC_TableEntryRead(bInitial = 0, pr = 1):

- ret = dict()
	
- ret["ret"] = sw.pyGSW_MAC_TableEntryRead(bInitial)

In [20]:
fs = open("Switch_d.py", "w")
fh = open("gsw_api_func_d.h", "r")
indentation = "          "

for line in fh.readlines():
    l = line.split()
    s = "def py" + l[0] + "("
    para = ""
    para_w = ""
    iter = 0
    if len(l) != 2 and l[2] == "0":
        iter = len(struct[l[1]])
        for i in range(iter):
            l.append(str(i))
    elif len(l) != 2 and l[2] == "2":
        iter = len(l)-3
    for i in range(iter):
        if len(s) > 100:
            s = s + "\\\n"
            fs.write(s)
            s = indentation
        ll = struct[l[1]][int(l[i+3])][1]
        intern = 1
        if ll[-1] == "]":
            intern = int(ll[ll.find("[")+1:-1])
            ll = ll[:ll.find("[")]
            s = s + ll + " = ["
            for n in range(intern):
                if len(s) > 100:
                    s = s + "\\\n"
                    fs.write(s)
                    s = indentation
                s = s + "0, "
            s = s[:-2] + "], "
        else:
            s = s + ll + " = 0, "

        for n in range(intern):
            if (len(para)) > 60:
                para = para + "\\\n"
                para_w = para_w + para
                para = indentation
            if intern == 1:
                para = para + ll + ", "
            else:
                para = para + ll + "[" + str(n) + "]" + ", "
    if (len(para)) >= 2:
        para = para[:-2]    
    s = s + "pr = 1):\n"
    s = s + "	ret = dict()\n"
    s = s + "	ret[\"ret\"] = sw.py" + l[0] + "(" + para_w + para + ")\n\n"

    if len(l) == 2 or l[2] == "0":
        s = s[:-1] + "	return ret\n\n"
        fs.write(s)
        continue
    s = s + "	fo = open(fn, \"r\")\n	for line in fo.readlines():\n		if len(line.split()) == 1:\n			line = line[:-1] + \"    ---    Return value: \" + nRet[ret[\"ret\"]] + \"\\n\"\n		else:\n			if line.split()[0][-1] == \"]\":\n				s = \"\"\n				for i in line.split()[2:]:\n					s = s + i + \" \"\n				ret[line.split()[0]] = s\n			else:\n				ret[line.split()[0]] = line.split()[3]\n		if pr == 1:\n			print (line[:-1])\n	fo.close()\n	return ret\n\n"
    fs.write(s)

fs.close()
fh.close()




In [120]:
d = dict()
d[f[0]] = f[3]
d

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [30]:
type(d)

dict

In [31]:
li=['a', 'b']   
li.append([2,'d'])   
li.append('e')   
li

['a', 'b', [2, 'd'], 'e']

In [32]:
type(li)

list

In [42]:
l = "/*dfklj/*lhjlj"
l[0:2]

'/*'

In [13]:
a = "mac[64]"
b1 = a.find("[")
c = int(a[b1+1:-1])
c

64